<a href="https://colab.research.google.com/github/Fermu25/Cursos/blob/main/Limpieza%20datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import re
import pandas as pd
from datetime import datetime, timedelta

# --------- ARCHIVOS ----------
archivo_entrada = "Tiempo de estabilización.txt"
archivo_salida  = "temperatura_estabilización.csv"

# --------- PATRONES ----------
# TCOR = número con punto o coma
re_tcor = re.compile(r"TCOR\s*=\s*([+-]?\d+(?:[.,]\d+)?)")

# 3 formas de tiempo posibles
p_hms_ms   = re.compile(r"\b(\d{1,2}:\d{2}:\d{2}(?:[.,]\d+)?)")                       # 20:46:37.371
p_hms_ampm = re.compile(r"\b(\d{1,2}:\d{2}:\d{2})\s*(a\.?\s*m\.?|p\.?\s*m\.?)", re.I) # 08:46:37 p. m.
p_ms_ms    = re.compile(r"\b(\d{1,2}:\d{2}(?:[.,]\d+)?)\b(?!:)")                      # 46:37.0

def parse_tiempo(linea):
    """Devuelve string normalizado y segundos relativos (los segundos los generaré luego si hace falta)."""
    m = p_hms_ms.search(linea)
    if m:
        return m.group(1).replace(',', '.')  # HH:MM:SS(.ms)

    m = p_hms_ampm.search(linea)
    if m:
        hhmmss, ampm = m.groups()
        ampm_clean = 'PM' if 'p' in ampm.lower() else 'AM'
        dt = datetime.strptime(f"{hhmmss} {ampm_clean}", "%I:%M:%S %p")
        return dt.strftime("%H:%M:%S")  # HH:MM:SS en 24h

    m = p_ms_ms.search(linea)
    if m:
        return m.group(1).replace(',', '.')  # MM:SS(.ms)

    return None

# --------- PARSEO ----------
tiempos = []
t1, t2, t3, t4 = [], [], [], []

total, usadas, descartadas = 0, 0, 0

with open(archivo_entrada, "r", encoding="utf-8", errors="ignore") as f:
    for linea in f:
        total += 1
        ts = parse_tiempo(linea)
        tcors = re_tcor.findall(linea)

        if ts and len(tcors) >= 4:   # por si a veces vienen más campos
            tiempos.append(ts)
            t1.append(float(tcors[0].replace(',', '.')))
            t2.append(float(tcors[1].replace(',', '.')))
            t3.append(float(tcors[2].replace(',', '.')))
            t4.append(float(tcors[3].replace(',', '.')))
            usadas += 1
        else:
            descartadas += 1

# --------- DATAFRAME ----------
df = pd.DataFrame({
    "Tiempo": tiempos,
    "T1": t1, "T2": t2, "T3": t3, "T4": t4
})

# (Opcional) convertir "Tiempo" a segundos transcurridos para graficar
def tiempo_a_segundos(s):
    # HH:MM:SS(.ms)
    if re.fullmatch(r"\d{1,2}:\d{2}:\d{2}(?:\.\d+)?", s):
        h, m, sec = s.split(":")
        return int(h)*3600 + int(m)*60 + float(sec)
    # MM:SS(.ms)
    if re.fullmatch(r"\d{1,2}:\d{2}(?:\.\d+)?", s):
        m, sec = s.split(":")
        return int(m)*60 + float(sec)
    return None

df["t_s"] = [tiempo_a_segundos(s) for s in df["Tiempo"]]
# normaliza a t=0 en la primera medición válida
if df["t_s"].notna().any():
    t0 = df["t_s"].dropna().iloc[0]
    df["t_s_rel"] = df["t_s"] - t0

# --------- GUARDAR ----------
df.to_csv(archivo_salida, index=False, encoding="utf-8")
print(f"Procesadas: {usadas}/{total} líneas. Descartadas: {descartadas}.")
print(f"CSV escrito: {archivo_salida}")
print(df.head())


Procesadas: 284/285 líneas. Descartadas: 1.
CSV escrito: temperatura_estabilización.csv
         Tiempo     T1     T2     T3     T4        t_s  t_s_rel
0  20:46:37.371  27.80  28.83  27.58  27.99  74797.371    0.000
1  20:46:38.878  27.80  28.83  27.52  27.99  74798.878    1.507
2  20:46:40.386  27.80  28.77  27.52  27.99  74800.386    3.015
3  20:46:41.893  27.80  28.77  27.52  28.12  74801.893    4.522
4  20:46:43.435  26.51  27.62  24.21  25.06  74803.435    6.064
